In [ ]:
!pip install datasets
!pip install torchmetrics
!pip install torch-fidelity

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset
from torchvision import transforms

from datasets import load_dataset, Image
import gc
import torch
from torchmetrics.image.fid import FrechetInceptionDistance

import random

In [ ]:
y = load_dataset("pcuenq/lsun-bedrooms")
#y = load_dataset("korexyz/celeba-hq-256x256")
#y = load_dataset("bitmind/ffhq-256")
#y_train = y['train'].cast_column("image", Image(mode="RGB"))
y_test = y['train'].cast_column("image", Image(mode="RGB"))
del y
gc.collect()
torch.cuda.empty_cache()

(…)-00000-of-00009-5beebd96eb33b02b.parquet:  13%|#2        | 62.9M/501M [00:00<?, ?B/s]

(…)-00001-of-00009-5f4afcffcee57481.parquet:   0%|          | 0.00/504M [00:00<?, ?B/s]

(…)-00002-of-00009-8f8c926d345e7432.parquet:   0%|          | 0.00/503M [00:00<?, ?B/s]

(…)-00003-of-00009-c9be5a4b95ca824c.parquet:   0%|          | 0.00/503M [00:00<?, ?B/s]

(…)-00004-of-00009-c985b2993da54f88.parquet:   0%|          | 0.00/501M [00:00<?, ?B/s]

...

(…)-00007-of-00009-87ce80047601018f.parquet:   0%|          | 0.00/503M [00:00<?, ?B/s]

(…)-00008-of-00009-926fe17e04968943.parquet:   0%|          | 0.00/502M [00:00<?, ?B/s]

(…)-00000-of-00001-7c2280a6897e2462.parquet:   0%|          | 0.00/238M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287968 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/15157 [00:00<?, ? examples/s]

In [ ]:
step=500

In [ ]:
x = np.load(f'/content/80x256x256x3-beds_{step}.npz')
all_samples = [torch.from_numpy(s)[torch.newaxis] for s in x['arr_0']]
images = torch.cat(all_samples, dim=0)
images = images.cpu()
images = images.to(torch.uint8)

In [ ]:
values_dict = {}
for f in [64, 192, 768, 2048]:

  metric = FrechetInceptionDistance(feature=f)
  values = []
  for _ in range(5):
    # sampling 80 photos
    sample_indices = np.random.choice(len(y_test), size=80, replace=False)
    sampled_images = [np.array(y_test[int(i)]['image'].resize((256,256))) for i in sample_indices]
    all_samples = [torch.from_numpy(i)[torch.newaxis] for i in sampled_images]
    real = torch.cat(all_samples, dim=0)
    real = real.cpu()
    real = real.to(torch.uint8)

    metric.update(real.permute(0,3,1,2), real=True)
    metric.update(images.permute(0,3,1,2), real=False)
    values.append(metric.compute())
    metric.reset()
  values_dict[f] = values
  #plt.figure(figsize=(10, 10))
  #fig, ax_ = metric.plot(values)
  #plt.title(f'FID for {f} features for LSUN bedrooms on test set')
  #plt.savefig(f'fid_{f}.png')
  #plt.close()
print(values_dict)

{64: [tensor(0.4845), tensor(0.4349), tensor(0.5418), tensor(0.4831), tensor(0.4957)], 192: [tensor(2.4171), tensor(1.9590), tensor(2.3202), tensor(2.9007), tensor(3.0730)], 768: [tensor(0.8660), tensor(0.8599), tensor(0.8328), tensor(0.8579), tensor(0.8550)], 2048: [tensor(95.7668), tensor(87.1624), tensor(86.4243), tensor(90.0622), tensor(91.0504)]}


In [ ]:
for k, v in values_dict.items():
  t = []
  for vv in v:
    t.append(vv.item())
  values_dict[k] = t

In [ ]:
values_dict

{64: [0.4845416843891144,
  0.4348525106906891,
  0.5417534112930298,
  0.48307737708091736,
  0.49567142128944397],
 192: [2.4171266555786133,
  1.9590096473693848,
  2.3202004432678223,
  2.900740385055542,
  3.0730133056640625],
 768: [0.8660065531730652,
  0.8598606586456299,
  0.8328036665916443,
  0.8578907251358032,
  0.8550111055374146],
 2048: [95.76683807373047,
  87.16239929199219,
  86.4243392944336,
  90.06219482421875,
  91.05035400390625]}

In [ ]:
import json
with open(f"beds_{step}.json", "w") as fp:
    json.dump(values_dict , fp)

In [ ]:
# prompt: find the average and std for each key

import numpy as np

def calculate_stats(values_dict):
  """Calculates the average and standard deviation for each key in a dictionary.

  Args:
      values_dict: A dictionary where keys are identifiers and values are lists of numbers.

  Returns:
      A dictionary with the same keys as the input, but with values as dictionaries containing 'average' and 'std'.
  """
  results = {}
  for key, values in values_dict.items():
      results[key] = {
          'average': np.around(np.mean(values), 3),
          'std': np.around(np.std(values), 3)
      }
  return results

# Calculate the statistics
stats_results = calculate_stats(values_dict)

# Print the results
print(stats_results)
# 50

{64: {'average': 0.488, 'std': 0.034}, 192: {'average': 2.534, 'std': 0.404}, 768: {'average': 0.854, 'std': 0.011}, 2048: {'average': 90.093, 'std': 3.321}}


In [ ]:
# Calculate the statistics
stats_results = calculate_stats(values_dict)

# Print the results
print(stats_results)

{64: {'average': 0.488, 'std': 0.034}, 192: {'average': 2.534, 'std': 0.404}, 768: {'average': 0.854, 'std': 0.011}, 2048: {'average': 90.093, 'std': 3.321}}
